<a href="https://colab.research.google.com/github/PinkPigmyPuff/SeatingChart/blob/main/ClassroomGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [241]:
import random
import numpy as np
from typing import List, Optional, Callable, Tuple


Chromosome = List[int]
Population = List[Chromosome]
Opinion = List[int]

In [242]:
def genChromosome(size: int) -> Chromosome:
    return random.sample(range(0, size), size)

def genPopulation(popSize: int, chromeosomeSize: int) -> Population:
    return [genChromosome(chromeosomeSize) for _ in range(popSize)]

def genOpinions(chromosome: Chromosome, opinionMax: int) -> Opinion:
  likes = []
  dislikes = []
  # for every student, randomly generate how many students they have opionions on, and add an empty array to like/dislike
  for i in range(0, len(chromosome)):
    likes.append([])
    dislikes.append([])
    likeNum = random.randint(0, opinionMax)
    dislikeNum = random.randint(0, opinionMax)
    dupelist = chromosome.copy()
    dupelist.pop(i)
    
    # choose random students that the current student likes
    for j in range(0, likeNum):
      randomStudent = random.randrange(0, len(dupelist))
      likes[i].append(dupelist[randomStudent])
      dupelist.pop(randomStudent)

    # choose random students that the current student dislikes
    for j in range(0, dislikeNum):
      randomStudent = random.randrange(0, len(dupelist))
      dislikes[i].append(dupelist[randomStudent])
      dupelist.pop(randomStudent)

  return likes, dislikes

In [243]:
def fitness(chromosome: Chromosome, peoplePerTable: int = 4, nextToFriendCost: int = 2, nextToEnemyCost: int = -1) -> int:
    size = len(chromosome)
    if size != len(likes) != len(dislikes):
        raise ValueError("genome and likes/dislikes must be of the same length")

    if size % peoplePerTable != 0:
        raise ValueError("The length of the chromosome must be divisable by peoplePerTable")
    # determine the cost of the given chromosome
    peopleHappiness = [0] * size
    chunkedClassroom = chunks(chromosome, peoplePerTable)
    # for every student
    for student in range(0, len(chromosome)):
      currentStudent = chromosome[student]
      studentsTable = 0
      # finds which table the currentStudent is sitting at
      if any(currentStudent in (match := nested_list) for nested_list in chunkedClassroom):
        studentsTable = chunkedClassroom.index(match)
        # print(f"current student is {currentStudent}, and they sit at table {studentsTable}")

      # determine the happiness of the currentStudent, based on each person at their table
      for tableMate in range(0, peoplePerTable):
        currentTableMate = chunkedClassroom[studentsTable][tableMate]
        if currentTableMate in likes[student]:
          peopleHappiness[student] += nextToFriendCost
        elif currentTableMate in dislikes[student]:
          peopleHappiness[student] += nextToEnemyCost

    return average(peopleHappiness)

def populationFitness(population: Population) -> float:
    return [fitness(chromosome) for chromosome in population]
    
# Return n-sized chunks from lst
def chunks(lst, n):
    x = [lst[i:i + n] for i in range(0, len(lst), n)]
    return(x)

def relu(x):
	  return max(0.0, x)
 
def average(lst):
   return float(sum(lst) / len(lst))

In [244]:
# Randomly chooses n individuals from the population and returns the fittest one
def tournamentSelection(population: Population, tournamentSize: int) -> Population:
		
		tournament = []
		tempPop = population.copy()
		for i in range(0, tournamentSize):
			random_id = random.randint(0, len(tempPop)-1)
			tournament.append(tempPop[random_id])
			tempPop.pop(i)

		#print(f"tournament pool! {tournament}")
		fittest = chooseHighest(tournament, 1)
		#print(f"fittest! {fittest}")
		return fittest

In [245]:
def roulette_wheel_selection(population: Population, populationFitness) -> Population:
    if len(population) != len(populationFitness):
        raise ValueError("genome and likes/dislikes must be of the same length")
    print(f"pFit: {populationFitness}")

    # Computes the totallity of the population fitness
    sumFitness = sum(populationFitness)
    print(f"sumFittness: {sumFitness}")
    
    # Computes for each chromosome the probability 
    chromosome_probabilities = [classroomHappiness/sumFitness for classroomHappiness in populationFitness]
    print(f"c_prob {chromosome_probabilities}")

    # Selects one chromosome based on the computed probabilities
    choice = population[np.random.choice(range(0, len(population)), p=chromosome_probabilities)]
    print(f"chioce_fitness {fitness(choice)}")
    return choice

In [246]:
def partiallyMatchedCrossover(ind1, ind2):
    #print(f"ind1: {ind1}\nind2: {ind2}")
    size = min(len(ind1), len(ind2))
    p1, p2 = [0] * size, [0] * size

    # Initialize the position of each indices in the individuals
    for i in range(size):
        p1[ind1[i]] = i
        p2[ind2[i]] = i
    # Choose crossover points
    cxpoint1 = random.randint(0, size)
    cxpoint2 = random.randint(0, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    # Apply crossover between cx points
    for i in range(cxpoint1, cxpoint2):
        # Keep track of the selected values
        temp1 = ind1[i]
        temp2 = ind2[i]
        # Swap the matched value
        ind1[i], ind1[p1[temp2]] = temp2, temp1
        ind2[i], ind2[p2[temp1]] = temp1, temp2
        # Position bookkeeping
        p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
        p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

    #print(f"FINSIHED, ind1: {ind1}\n ind2: {ind2}")
    return ind1, ind2

In [247]:
def mutate(chromosome: Chromosome) -> Chromosome:
    length = len(chromosome)
    if length < 2:
      print("Chromosome is too short to be mutated")
      return chromosome
    index1, index2 = random.sample(range(0, length), 2)
    chromosome[index1], chromosome[index2] = chromosome[index2], chromosome[index1]
    #print(f"mutated chromosome: {chromosome}")
    return chromosome

In [248]:
def chooseHighest(population: Population, numToPick: int = 1):
  tempPop = population.copy()
  fit = populationFitness(tempPop)
  fittest = []
  for num in range(numToPick):
      highestIndex = 0
      for i in range(0, len(fit)):
          if fit[i] > fit[highestIndex]:
              highest = fit[i]
              highestIndex = i
      if numToPick == 1:
          return tempPop[numToPick]
      fittest.append(tempPop[highestIndex])
      tempPop.pop(highestIndex)
  return fittest

In [249]:
pop = genPopulation(40, 40)
likes, dislikes = genOpinions(pop[0], 3)

def run_evolution(
        generation_limit: int = 30,
        population = genPopulation(8, 8), 
        #likes = genOpinions(population[0], 4),
        #dislikes = genOpinions(population[0], 4)
        #peoplePerTable = 4,
        #nextToFriendCost = 1,
        #nextToEnemyCost = -1,
        ):
    print(f"OG population: {population}")
    print(f"Likes: {likes}\nDislikes: {dislikes}")
    if len(population) % 2 != 0:
        print("Population size is odd, removing one chromosome")
        population.pop(0)


    for i in range(generation_limit):
        nextGeneration = []

        # Find and print the fitness of the population
        popFitness = populationFitness(population)
        print(f"\n\nGeneration {i} fitness: {popFitness}")
        print(f"population: {population}")


        parents = [tournamentSelection(population, 4) for i in range(len(population))]
        print(f"parents: {parents}")

        #random.shuffle(parents)
        while parents:
          #print(f"b0: {parents[0]}\nb1: {parents[1]}")
          offspring1, offspring2 = partiallyMatchedCrossover(parents[0].copy(), parents[1].copy())
          nextGeneration.append(mutate(offspring1.copy()))
          nextGeneration.append(mutate(offspring2.copy()))          
          parents = parents[2:]

        # Set the current population to equal the kids
        population = nextGeneration.copy()

    return chooseHighest(population, 3)


top3 = run_evolution(population=pop)
print(populationFitness(top3))
print(top3)

OG population: [[23, 14, 15, 21, 7, 35, 34, 28, 0, 30, 8, 6, 26, 11, 3, 29, 10, 4, 25, 2, 27, 22, 33, 32, 5, 12, 19, 38, 20, 39, 16, 9, 36, 1, 13, 37, 24, 31, 18, 17], [12, 4, 8, 18, 15, 39, 24, 11, 6, 38, 30, 5, 37, 23, 28, 33, 2, 13, 29, 16, 9, 3, 26, 36, 1, 19, 21, 27, 14, 35, 20, 32, 10, 0, 34, 17, 31, 25, 7, 22], [3, 29, 8, 14, 13, 16, 17, 19, 7, 37, 33, 35, 18, 27, 32, 5, 15, 11, 21, 38, 30, 9, 31, 23, 2, 36, 22, 26, 25, 20, 0, 12, 4, 24, 34, 10, 6, 28, 39, 1], [0, 13, 7, 16, 5, 9, 28, 18, 12, 29, 8, 34, 36, 37, 30, 14, 15, 19, 23, 10, 35, 4, 32, 21, 1, 26, 2, 24, 27, 6, 38, 3, 25, 33, 17, 31, 39, 11, 20, 22], [6, 30, 21, 25, 36, 9, 15, 0, 22, 29, 28, 4, 18, 14, 16, 3, 10, 27, 1, 34, 11, 35, 5, 33, 19, 12, 7, 23, 26, 17, 37, 20, 39, 2, 8, 13, 38, 31, 32, 24], [29, 30, 38, 14, 6, 24, 1, 21, 23, 7, 15, 34, 33, 37, 10, 5, 27, 22, 12, 0, 8, 25, 13, 2, 32, 26, 19, 11, 9, 18, 20, 31, 16, 36, 3, 39, 28, 4, 35, 17], [22, 38, 10, 18, 11, 15, 34, 9, 2, 39, 13, 28, 27, 21, 16, 14, 24, 20, 4

1. Encoding scheme
- A chromosome is a permutation list
- Each index is an integer
- Each integer is a student in the class

2. Create the initial population
- First, a function to create a single chromsome
  - inputs chromosomeSize
  - outputs a chromosome of randomly shuffled integers

7. Testing
- A func